## Aula 8 - Agregação com Pandas, Ambiente de Desenvolvimento e Git

### Agregação com Pandas

In [1]:
import pandas as pd
import numpy as np

#### Exemplo de Group By com Pandas

In [2]:
# Criando um dataframe
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.143751,0.333226
1,a,two,0.118217,-0.392849
2,b,one,-1.203912,-1.039238
3,b,two,1.279386,0.392964
4,a,one,1.281998,-0.482165


In [ ]:
# Criar um objeto de agrupamento da coluna data1 agrupando por key1
grouped = df['data1'].groupby(df['key1'])
# Veja que o resultado retornado é um objeto Pyhton SeriesGroupBy 
grouped

In [ ]:
# Aplicando a métrica Média (mean) sobre esse objeto de agrupamento
# Perceba que a média sobre o objeto SeriesGroupBy vai retornar uma Series como resultado,
#   e o índice será a coluna de agrupamento key1
grouped.mean()

#### Agregação com duas colunas

In [ ]:
# Criar um objeto de agrupamento da coluna data1 agrupando por key1 e key2
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
# Veja que means é uma Series com índices key1 e key2
means

In [ ]:
# Curiosidade: você pode pivotar essa série com o médoto unstack()
means.unstack()

#### Aplicando a métrica em todas as colunas de dados

In [ ]:
# Agrupe o dataframe df por key1 e extraia a média de todas as colunas
df.groupby('key1').mean()

In [ ]:
# perceba que o objeto retornado pelo agrupamento do dataframe inteiro é 
#   um DataFrameGroupBy 
df.groupby('key1')

In [ ]:
# Agrupe o dataframe inteiro pelas colunas key1 e key2 calculando o valor da média
#    para as outras colunas.
df.groupby(['key1', 'key2']).mean()

#### Mostrando o tamanho dos grupos

In [ ]:
# Mostre o tamanho dos grupos de um agrupamento do dataframe df pelas colunas key1 e key2
df.groupby(['key1', 'key2']).size()

#### Agrupando com uma função

In [3]:
# Considere o seguinte dataframe
people = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'], 
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Adicionando alguns valores NA
people

,a,b,c,d,e
Joe,-0.487419,0.395697,0.030361,-0.913176,0.798658
Steve,-1.364818,0.735031,0.460231,-0.201578,-1.396227
Wes,-0.373420,NaN,NaN,-0.084181,0.271106
Jim,-2.235506,0.310970,0.681183,-1.103456,-0.783627
Travis,0.012762,-0.368397,0.789107,0.350320,-1.816616


In [4]:
# Agrupe o dataframe people pelo tamanho das strings do indice retornando a soma de cada
#    grupo.
people.groupby(len).sum()

,a,b,c,d,e
3,-3.096344,0.706667,0.711544,-2.100813,0.286137
5,-1.364818,0.735031,0.460231,-0.201578,-1.396227
6,0.012762,-0.368397,0.789107,0.350320,-1.816616


#### Aplicando várias métricas ao mesmo tempo

In [5]:
# Considere o seguinte dataset de gorgetas (tips)
import pandas as pd
import io # Biblioteca com funções de IO (dentre elas de Strings)
import requests #Biblioteca para fazer requisições HTTP
url="https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv"
s=requests.get(url).content
tips=pd.read_csv(io.StringIO(s.decode('utf-8')))
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [6]:
# Adiciona uma coluna para calcular a porcentage da gorjeta em relação ao valor da conta
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [7]:
# Agrupar por dia da semana (day) e fumante (smoker)
grouped = tips.groupby(['day', 'smoker'])
grouped

In [11]:
# Pegar um slice (uma fatia) do objeto de agrupamento com apenas a coluna tip_pct
grouped_pct = grouped[['tip_pct', 'total_bill']]
grouped_pct

In [12]:
# Calcular a média deste agrupamento
grouped_pct.agg('mean')

tip_pct  total_bill
day  smoker                      
Fri  No      0.151650   18.420000
     Yes     0.174783   16.813333
Sat  No      0.158048   19.661778
     Yes     0.147906   21.276667
Sun  No      0.160113   20.506667
     Yes     0.187250   24.120000
Thur No      0.160298   17.113111
     Yes     0.163863   19.190588

In [13]:
# Considere a função para retornar o intervalo de variação de um array
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [24]:
# Escreva numa só sentença a aplicação das métricas média, desvio padrão (std) 
#   e peak_to_peak ao slice do agrupamento criado acima
pk = lambda x: x.max() - x.min()
grouped_pct.agg({'total_bill': ['mean', pk]})

total_bill                                 
               tip_pct            total_bill           
                  mean <lambda_0>       mean <lambda_0>
day  smoker                                            
Fri  No       0.151650   0.067349  18.420000      10.29
     Yes      0.174783   0.159925  16.813333      34.42
Sat  No       0.158048   0.235193  19.661778      41.08
     Yes      0.147906   0.290095  21.276667      47.74
Sun  No       0.160113   0.193226  20.506667      39.40
     Yes      0.187250   0.644685  24.120000      38.10
Thur No       0.160298   0.193350  17.113111      33.68
     Yes      0.163863   0.151240  19.190588      32.77

#### Várias métricas em diferentes colunas

In [20]:
# Usando um dicionário para mapear as colunas e suas respectivas métricas, aplique
#     à coluna tip_pct as métricas 'min', 'max', 'mean', 'std' e 
dfr = grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})
dfr.reset_index()

day smoker   tip_pct                               size
                     min       max      mean       std  sum
0   Fri     No  0.120385  0.187735  0.151650  0.028123    9
1   Fri    Yes  0.103555  0.263480  0.174783  0.051293   31
2   Sat     No  0.056797  0.291990  0.158048  0.039767  115
3   Sat    Yes  0.035638  0.325733  0.147906  0.061375  104
4   Sun     No  0.059447  0.252672  0.160113  0.042347  167
5   Sun    Yes  0.065660  0.710345  0.187250  0.154134   49
6  Thur     No  0.072961  0.266312  0.160298  0.038774  112
7  Thur    Yes  0.090014  0.241255  0.163863  0.039389   40